# Importing

In [1]:
import pandas as pd
import numpy as np

from gensim.models import FastText

from tqdm.notebook import tqdm

# Data Loading

## Raw Data
자기가 쓰는 path에 알아서 맞춰서

In [4]:
train = pd.read_json('../res/train_token_full.json')
val = pd.read_json('../res/val_token_full.json')
test = pd.read_json('../res/test_token_full.json')

## Preprocess

In [2]:
def make_sentence(df):
    ttl = df.ttl_token_freq.apply(lambda x: [e[0] for e in x])
    tag = df.tag_token_freq.apply(lambda x: [e[0] for e in x])
    return ttl + tag

In [13]:
def to_one_string(lst):
    tmp = ''
    for i, word in enumerate(lst):
        if i == len(lst)-1:
            tmp += word
        else:
            tmp += word + ' '
    return tmp

In [5]:
train_words = make_sentence(train)
val_words = make_sentence(val)

In [7]:
token_dict = dict()
for lst in train_words:
    for word in lst:
        try:
            token_dict[word] += 1
        except:
            token_dict[word] = 1
            
print("# of Distinct words: {}".format(len(token_dict)))

# of Distinct words: 46575


In [8]:
how_many = pd.Series([v for v in token_dict.values()], index=[k for k in token_dict.keys()])

## Wiping out

In [10]:
general = set(['음악', '곡', '노래', '모음', '때'])
not_used = set(how_many[how_many <= 2].index)
deletion = general.union(not_used)
deletion

{'ju',
 '3456',
 '머리좀',
 'heard',
 'saaa',
 'leebadaSpecial',
 '대체누',
 'Worry',
 '겨울밤이불안',
 '때매',
 '파워워킹할때',
 '라이데',
 '주말드라',
 '상배경',
 '지소중',
 '자리매김',
 '선경감자많',
 '열정가득',
 '어쩌',
 'SK',
 '케이아',
 '필사',
 '아시',
 '팝쟁',
 '감성자극곡',
 'shuffle',
 '업무중',
 '띵곡띵곡',
 '피로야가',
 '아담리바',
 '조쟈',
 '열반',
 '호구맨',
 '인스턴트',
 '시켜',
 'trench',
 '대격돌',
 '블레',
 '요국내편',
 'Piaf',
 '영익이',
 '지지배',
 '20141003',
 '리푸쓰',
 'SweetSweetSweet',
 '라드감',
 '그루브그루브',
 '둠칫둠',
 '단어만',
 '음반악기',
 '140425',
 '포비트감성',
 '1042',
 '부시',
 '옷차림',
 '송가인채널',
 'wagle',
 '레코드판',
 '톡으로',
 '캠핑족',
 '인래퍼',
 '보밀꺼',
 '새큼달',
 'TODD',
 '명품가요발라드',
 '돌히',
 '차이콥',
 '매일밤',
 '흥흥흥',
 '조오',
 '유향',
 'Rossi',
 '미담',
 '버즈남정네',
 '디로',
 'TECHSHT',
 '비가오는밤',
 'Ros',
 '찰떡노',
 '지치지',
 '조덕배',
 '솔로듀',
 '에릭킴스토리',
 'clupdancecome',
 '썸에',
 '증',
 '켄타',
 '아이딜',
 '태양빛',
 '국힙곡',
 '고층',
 '노래어때',
 '착플리',
 '낌트렌',
 '아지트',
 '방댕이',
 '자뻑',
 '명수',
 '오르간연주자',
 '툭하',
 'Still',
 '만성피로',
 '겨우',
 'GMA',
 'prada',
 '망설임',
 '지모',
 '옛날느낌',
 '인디인',
 'Pinec',
 '꼬마버스',
 '790',
 'GRAET',
 'Fra

In [14]:
# 학습시킬 단어들
train_words = train_words.apply(lambda x: list(set(x) - deletion))

# 그 단어들 하나의 string으로 뭉쳐놓은것
train_str = train_words.map(to_one_string)

In [17]:
val_ttl_only = val.loc[(np.array(list(map(len, val.songs))) == 0) & (np.array(list(map(len, val.tags))) == 0)]

val_words = val_ttl_only.ttl_token_freq.map(lambda x: [e[0] for e in x])
val_words = val_words.apply(lambda x: list(set(x) - deletion))
val_str = val_words.map(to_one_string)

In [20]:
val_ttl_only.plylst_title

1                                   앨리스테이블
9                  기분 좋은 재즈와 함께 만드는 달달한 하루
35                                사랑그리고이별 
57                         마쉬멜로우같은 멜로우한 음악
71                       공부와 독서를 위한 Newage
                       ...                
22903             10년이 지나 들어도 좋은 감성 Ballad
22920        가사의 의미와 뜻은모른다 오직 멜로디로만 선곡한 팝송
22981                             카페 느낌 샹송
22991    컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작
22996                              해 저무는 밤
Name: plylst_title, Length: 1749, dtype: object

# FastText Train

In [21]:
from gensim.models import FastText
model = FastText(train_words, window=2)

In [25]:
def get_set(series):
    song_set = dict()
    for lst in series:
        for songs in lst:
            try:
                song_set[songs] += 1
            except:
                song_set[songs] = 1
                
    return song_set

In [26]:
def predict(target, verbose=True):
    
    if verbose:
        print(target, val.plylst_title[target])
    
    ids = val.id[target]
    err = False
    
    sims = [(i, model.wv.similarity(val_str[target], ttl)) for i, ttl in enumerate(train_str)]
    result = sorted(sims, key=(lambda x: x[1]))
    
    # Song Part
    k = 20
    song_set = dict()
    while len(song_set) < 100:
        mask = [r[0] for r in result[::-1][:k]]
        song_set = get_set(train.loc[mask, 'songs'])
        if len(song_set) < 100:
            k += 20
            
    similar_songs = sorted(song_set.items(), key=(lambda x: x[1]))
    pred_songs = similar_songs[::-1][:100]
    if len(pred_songs) < 100:
        err = True
        print("ID {} has Track less than 100".format(ids))
    
    # Tag Part
    k = 20
    tag_set = dict()
    while len(tag_set) < 10:
        mask = [r[0] for r in result[::-1][:k]]
        tag_set = get_set(train.loc[mask, 'tags'])
        if len(tag_set) < 10:
            k += 20
            
    tag_similarity = [(t, model.wv.similarity(val_str[target], t)) for t in tag_set.keys()]
    similar_tags = sorted(tag_similarity, key=(lambda x: x[1]))
    pred_tags = similar_tags[::-1][:10]
    if len(pred_tags) < 10:
        err = True
        print("ID {} has Tags less than 10".format(ids))
        

    return ids, pred_songs, pred_tags, err

## Debug

In [27]:
predict(199, True)

199 기분전환 KPOP 뮤직


(67281,
 [(470129, 5),
  (236393, 3),
  (119974, 3),
  (660013, 3),
  (551411, 2),
  (24087, 2),
  (153130, 2),
  (223252, 2),
  (65557, 2),
  (14450, 2),
  (263588, 2),
  (222305, 2),
  (360825, 2),
  (253594, 2),
  (257920, 2),
  (157861, 2),
  (639051, 2),
  (542127, 2),
  (119795, 2),
  (439301, 2),
  (323755, 2),
  (582372, 2),
  (352212, 2),
  (103189, 2),
  (175418, 2),
  (15124, 2),
  (69648, 2),
  (309890, 2),
  (101477, 2),
  (394489, 2),
  (579817, 2),
  (119878, 2),
  (642897, 2),
  (90630, 2),
  (304426, 2),
  (318762, 2),
  (606964, 2),
  (403537, 2),
  (62174, 2),
  (376435, 2),
  (429998, 2),
  (545693, 2),
  (176435, 2),
  (77071, 2),
  (701801, 2),
  (310151, 2),
  (441702, 2),
  (72222, 2),
  (137421, 2),
  (567194, 2),
  (195174, 2),
  (43627, 2),
  (581799, 2),
  (559607, 2),
  (217365, 2),
  (538860, 2),
  (473639, 2),
  (32120, 2),
  (19096, 2),
  (286015, 1),
  (318945, 1),
  (687936, 1),
  (417405, 1),
  (607839, 1),
  (270632, 1),
  (36077, 1),
  (71100, 1),
 

In [23]:
pred, log = [], []

In [ ]:
for i in tqdm(val_str.index):
    
    ids, s, t, err = predict(i)
    pred.append({
        "id": ids,
        "songs": list(map(lambda x: x[0], s)),
        "tags" : list(map(lambda x: x[0], t))
    })
    print(t)
    if err:
        log.append(ids)

1 앨리스테이블
9 기분 좋은 재즈와 함께 만드는 달달한 하루
35  사랑그리고이별 
57 마쉬멜로우같은 멜로우한 음악
71 공부와 독서를 위한 Newage
101 새벽에 감성 돋는 노래
115 인디 OST 음악을 만나다 50
135 너 없이 살수가 없어그리움
167 귀를 달달하게 만들어주는 남녀 듀엣곡
199 기분전환 KPOP 뮤직
218 냉정과 열정사이 가을
220 나른한 오후 카페에 앉아 커피를 마시며 듣는 연주곡
221 발라드드
240 후기 바로크 1117세기후18세기중바흐4
253 신나는 곡좋아하는 곡
254 흔하지 않아서 더 특별한 여성 보컬 모음곡
257 클래식과 함께하는 퇴근길
279 봄봄봄봄봄이왔어요
300 취향저격 팝송
304 구름을 걷는 듯한 솜사탕같은 제이팝
307 뜨거운 햇살 아래 시원한 그늘 같은 여성 보컬
308 주말을 마무리하며 듣는음악
317 신해경이라는 장르로 힐링하기
333 5회 아시아송페스티벌
334 위로가 되어주는 인디
352 좋아하는 노래
379 매혹적인 아카펠라 재즈
384 달달보스
391 락페스티벌이 생각나는 국내 밴드음악
400 밤에 찾게 되는 알앤비 소울 
416 비오는 날 듣기좋은 음악
420 라디오 972
430 마음이 꽃피는 하루
440 방탄잠잘때듣는노래
454 가을하고 어울리는 감성 Rock
466 혼자감당해야 하는거니
474 UK 프로젝트 vol1
475 선선한 바람 맞으며 산책할 때 듣는 음악
481 이별의 끝은
490 시원한 노래 모음집
498 IMJMWEDAPLUGG
540 첫 도입부분부터 고혹적인 음악
547 클래식으로 맞이하는 저녁
572 숨겨져 있던 좋은 알앤비힙합
598 슬픔이 차오를때 듣는 팝
601 My POP 2
613 잔잔한발라드19곡
615 마음만은 뉴요커인 당신에게
633 추석 특선 영화 귀로 듣는다
648 클럽뮤직 요즘 거로 듣자
675 1990년대 발라드
680 지치고 힘들때 힘이 되주는 힐링곡
681 추억의 90년대 댄스가요
683 푹 빠지게되는 감성적인팝송들
693 soul감

5006 있는그대로의 인디
5029 일렉
5102 추억속에 민들레 되어
5109 드라이빙 Or 워킹
5115 편하게듣기좋은사랑노래
5119 꿈꾸는 아기에게 들려주고픈 피아노
5120 즐겨찾기 13 여자솔로3
5138 여유로움 끝판왕 POP 띵곡
5146 스트레스 해소 Or 감성충전소 33
5161 이열치열 뜨거운 인디음악과 아이스크림
5164 가장 그다웠던 공연 문문 THANK YOU 곰곰
5176 밤 몽환적이고 무드있는 노래들
5200 스트레스엔 역시 펑크락
5236 팝음악수집 06월
5274 비오는 날에는 JAZZ
5290 골방뮤직 독립만세 56
5295 레드벨벳 수능 금지곡
5307 크리스마스 따뜻한 감성의 캐롤 팝과 올드팝
5331 아이씐나씐나는 노래 
5334 주님이 내게
5340 GIMME ME MORE COFFEE
5347 즐거운 재즈 와 함께 더위 탈출
5354 하나의 이야기
5355 드라이브와 함께하는 아름다운 음악
5356 취침전에 듣는 아이돌 노래 10
5372 이 달의 노래
5373 드라이브할때
5384 When October Goes에 마주친 재즈 발라드
5397 하루를 마무리 할 때 들어봐요
5409 소울 넘치는 정통 흑인음악 RB
5416 추워질수록 생각나는 첼로선율들
5420  지독한 가을앓이에 생채기 낼 음악
5423 2013년 나만의 올해의 앨범과 곡들
5436 추억을 만들기 위해서
5438 센치한밤에 듣는 분위기있는 음악들
5452 뉴에이지 최신곡 모음집 2015년 07월 24일
5474 클래식 최신곡 모음집 2015년 10월 28일
5488 요즘 빠진노래들개취주의
5491 아 이거 무슨 드라마였지 OST 플레이리스트
5506 취향저격 팝송
5514 봄의 피크닉
5518 Calm
5522 추억에 잠기고싶을때 분위기에 취하고싶을때
5525 라디오 363
5530 비 오는 날의 버스
5571 유튜브 100만구독 1MILLION Dance Studio노래 모음
5573 오늘의 추천 가수 어쿠루브
5574 거짓말같은 날
5576 겨울